In [1]:
from py2neo import Graph

In [2]:
# Connect to your Neo4j database
neo4j_uri = "bolt://localhost:7687"  # Replace with your Neo4j server URI
#neo4j_user = "your_username"  # Replace with your Neo4j username
#neo4j_password = "your_password"  # Replace with your Neo4j password

graph = Graph(neo4j_uri)

ConnectionUnavailable: Cannot open connection to ConnectionProfile('bolt://localhost:7687')

In [ ]:
# Run the Cypher query to retrieve all nodes
def get_all_nodes():
    query = "MATCH (node) RETURN node"
    result = graph.run(query)

    # Fetch and display the results
    #for record in result:
        #print(record)
    #print(result)
    return result

In [ ]:
# Run the Cypher query to retrieve all relations
def get_all_relations():
    query = "MATCH (start)-[r]->(end) RETURN start, r, end"
    result = graph.run(query)

    # Fetch and display the results
    #for record in result:
        #print(record)
    #print(result)
    return result

In [ ]:
def check_relation(node_a, node_b): 
    query = (
    "MATCH (a)-[r]->(b) "
    "WHERE a.name = $node_a AND b.name = $node_b "
    "RETURN COUNT(r) AS count "
    )

    result = graph.run(query, node_a=node_a, node_b=node_b).data()

    # Check if the relationship exists (count > 0)
    if result[0]['count'] > 0:
        print(f"Relationship exists between nodes {node_a} and {node_b}.")
        return True
    else:
        print(f"No relationship found between nodes {node_a} and {node_b}.")
        return False
    

    #result = graph.run(query)

    # Fetch and display the results
    #for record in result:
        #print(record)
    #print(result)
    
    #return result

In [ ]:
def add_rel(node_a, node_b, occur):

    # Relationship type and property
    #relationship_type = "IS_CONNECTED"  # Replace with the desired relationship type

    # Cypher query to create a new relationship with a "count" property
    query = (
        "MATCH (a), (b) "
        "WHERE a.name = $node_a AND b.name = $node_b "
        "CREATE (a)-[r:IS_CONNECTED]->(b) "
        "SET r.count = $occur "
        "RETURN r"
    )

    result = graph.run(query, node_a=node_a, node_b=node_b, occur=occur).data()

    # Check the result or print a message
    if result:
        print(f"Created new IS_CONNECTED relationship between nodes '{node_a}' and '{node_b}' with 'count' property set to {occur}.")
    else:
        print(f"Nodes with names '{node_a}' and '{node_b}' not found.")

In [ ]:
def add_occur_to_rel(node_a, node_b, occur):
    # Relationship type
    #relationship_type = "IS_CONNECTED"  # Replace with the actual relationship type

    # Cypher query to add the number to the "count" property of the relationship
    query = (
        "MATCH (a)-[r:IS_CONNECTED]->(b) "
        "WHERE a.name = $node_a AND b.name = $node_b "
        "SET r.count = COALESCE(r.count, 0) + $occur "
        "RETURN r"
    )

    result = graph.run(query, node_a=node_a, node_b=node_b, occur=occur).data()

    # Check the result or print a message
    if result:
        print(f"Added {occur} to the 'count' property of the IS_CONNECTED relationship between nodes '{node_a}' and '{node_b}'.")
    else:
        print(f"Relationship not found between nodes '{node_a}' and '{node_b}'.")

In [ ]:
def add_occur_to_node(node, number):
    # Cypher query to add the number to the "occur" property
    query = (
        "MATCH (a) WHERE a.name = $node "
        "SET a.occur = COALESCE(a.occur, 0) + $number "
        "RETURN a"
    )

    result = graph.run(query, node=node, number=number).data()

    # Check the result or print a message
    if result:
        print(f"Added {number} to the 'occur' property of node '{node}'.")
    else:
        print(f"Node with name '{node}' not found.")

In [ ]:
def add_inverted_relations():
    
    relations = list(get_all_relations())

    for record in relations:
        start_node = record["start"]
        end_node = record["end"]
        relationship = record["r"]
        relation_start_node = start_node.get("name")
        relation_end_node = end_node.get("name")
        relation_count = relationship.get('count')

        print(f"Start Node: {relation_start_node}, End Node: {relation_end_node}, Count: {relation_count}")
        if (check_relation(relation_end_node, relation_start_node)):
            print("Relation found!")
            add_occur_to_rel(relation_start_node, relation_end_node, relation_count)
        else:
            print("Relation not found, adding a new one!")
            add_rel(relation_end_node, relation_start_node, relation_count)
        
        print("Adding occurences to nodes")
        add_occur_to_node(relation_start_node, relation_count)
        add_occur_to_node(relation_end_node, relation_count)
        
    print("ALL DONE!")
    
        # Write the output file
        #output_filename = f'Relationships.txt'
        #with open(os.path.join(abs_folder_path, output_filename), 'a', encoding="utf-8") as output_file:
        #    output_file.write(f"|{relation_start_node}|{relation_end_node}|{relation_count}|\n")



In [ ]:
add_inverted_relations()